In [1]:
#load the data

import csv
import pandas as pd
import numpy as np
import scipy as sp 
import math

with open('ENSG00000013016_EHD3_NT.table.dat', 'r') as file:
    table = pd.read_csv(file, delimiter=',', header = None)

with open('ENSG00000013016_EHD3_NT.msa.dat', 'r') as file:
    msa = pd.read_csv(file, delimiter=',', header = None)

with open('ENSG00000013016_EHD3_NT.branchlength.dat', 'r') as file:
    brlen = pd.read_csv(file, delimiter=',', header = None)

with open('ENSG00000112984_KIF20A_NT.table.dat', 'r') as file:
    table2 = pd.read_csv(file, delimiter=',', header = None)

with open('ENSG00000112984_KIF20A_NT.msa.dat', 'r') as file:
    msa2 = pd.read_csv(file, delimiter=',', header = None)

with open('ENSG00000112984_KIF20A_NT.branchlength.dat', 'r') as file:
    brlen2 = pd.read_csv(file, delimiter=',', header = None)
    
with open('table.dat', 'r') as file:
    table3 = pd.read_csv(file, delimiter=',', header = None)

with open('msa.dat', 'r') as file:
    msa3 = pd.read_csv(file, delimiter=',', header = None)

with open('branchlength.dat', 'r') as file:
    brlen3 = pd.read_csv(file, delimiter=',', header = None)

# these lines are to split the columns in the msa files into a columns with the name of the species and the sequence itself
msa[[0,1]] = msa[0].str.split(' ', expand=True)
msa2[[0,1]] = msa2[0].str.split(' ', expand=True)
msa3[[0,1]] = msa3[0].str.split(' ', expand=True)

Q = np.array([[-0.5625,0.1875,0.1875,0.1875],[0.1875,-0.5625,0.1875,0.1875],
 [0.1875,0.1875,-0.5625,0.1875],[0.1875,0.1875,0.1875,-0.5625]])



In [2]:
# creation of the function to read the raw data and format them into a list of Node class objects (that we will define later)
def create_node_list(tab, ms, brl):
    node_list = []
    nodes = {}
    branches = iter(brl.iloc[0]) # we use an iterator to go through the branch lengths and add them as an attribute to the nodes
    
    for i in range(len(tab[1])): # we go through the children column if the table data to create the nodes except for the root node
            
        itself = str(tab[1][i])
        parent =  tab[0][i]
        sib_index = [index for index, value in enumerate(tab[0]) if value == parent and str(tab[1][index]) != itself] # we get the index of the sibling of the node
        sibling = str(tab[1][sib_index[0]]) # we get the sibling of the node
        branch = next(branches)
        sequence = np.array(list(ms.loc[ms[0] == itself, 1].iloc[0])) if itself in ms[0].values else None # if the node is in the msa data, we get the sequence
        root = False       
        nodes[itself] = Node(itself, str(parent), sibling, sequence, branch, root) # we define the nodes with the currently known attributes
        node_list.append(nodes[itself])
    for i in tab[0]: # we go through the parent column to find the root node, so the node that is not in the children column

        if str(i) not in [str(x) for x in list(tab[1])]:
            itself = str(i)
            parent =  None
            sibling = None
            branch = None
            root = True
            sequence = np.array(list(ms[1][itself])) if itself in ms[0] else None
            nodes[itself] = Node(itself, parent, sibling, sequence, branch, root)
            node_list.append(nodes[itself])
            break
    return node_list

def find_node_by_number(node_list, number): # function to find a node by its number in the node list and return the object node
    for node in node_list:
        if node.number == number:
            return node
    return None

def find_node_by_sigling(node_list, number): # function to find a node by its sibling in the node list and return the object node
    for node in node_list:
        if node.number == find_node_by_number(node_list, number).sibling and node.number is not None:
            return node
    return None
def find_node_by_parent(node_list, number): # function to find a node by its children in the node list and return the object node 
    for node in node_list:
        if node.number == find_node_by_number(node_list, number).parent:
            return node
    return None
def find_root_node(node_list): # function to find the root node in the node list and return the object node
    for node in node_list:
        if node.root == True:
            return node
    return None

In [3]:
# here we defined the Node class and the Tree class

class Node:
    def __init__(self, number, parent, sibling, seq, length, root):
        self.number = number
        self.parent = parent
        self.sibling = sibling
        self.seq = seq
        self.length = length
        self.root = root

    def add_probability_matrix(self, matrix): # function within the Node class to add the probability matrix to the node based on the branch length and the Q matrix
        if hasattr(self, 'length') and self.length is not None: # the function checks if the node has a branch length attribute
            self.matrix = sp.linalg.expm(self.length * matrix)
        else:
           # print("No length attribute found")
            self.matrix = None
    def add_seq_probability(self): # function within the Node class to add the probability of the sequence to the node based on the sequence and the probability matrix
        base_prob = {"A": np.array([1,0,0,0]), "T":np.array([0,1,0,0]), "C" : np.array([0,0,1,0]), "G" : np.array([0,0,0,1])} # dictionary to map the base to the probability vector
        map_values = np.vectorize(lambda x: base_prob[x], signature='()->(n)') # function to map the base to the probability vector
        if hasattr(self, 'seq') and self.seq is not None: # the function checks if the node has a sequence attribute and maps the sequence to the probability vector
            self.prob = map_values(self.seq)
        else:
            #print("No sequence attribute found")
            self.prob = None

class Tree: # class to define the tree and its methods
    def __init__(self, position): # position is the attribute of the tree that is a list of Node objects
        self.position = position
    def fill_probability_matrix(self, matrix): # function to fill the probability matrix of the nodes in the tree using the method defined in the Node class
        for node in self.position:
            node.add_probability_matrix(matrix)
            node.add_seq_probability()
    def fill_probability_sequence(self): # function to fill the probability sequence of the nodes in the tree using the method defined in the Node class
        root_node = find_root_node(self.position)
        b = len(self.position)
        a = 0
        while a <= b: # normally the loop should stop when the root node has a probability sequence but in case the data is not well formatted, we put a limit to the number of iterations
            for node in self.position:
                sib = find_node_by_sigling(self.position, node.number)
                parent = find_node_by_parent(self.position, node.number)
                if sib == None or parent == None:
                    continue
                if  node.root == False and node.prob is not None and sib.prob is not None and parent.prob is None: # we check if the parental node of two sibling with a prob seq has a probability sequence and if not we calculate it
                    vec1 = np.matmul(node.prob, node.matrix)
                    vec2 = np.matmul(sib.prob, sib.matrix)
                    vec_parent = vec1 * vec2
                    parent.prob = vec_parent
                else:
                    continue
            if root_node.prob is not None: # we check if the root node has a probability sequence and if it does we break the loop
                break
            a += 1

    def tree_log_likelihood(self, matrix): # function to calculate the log likelihood of the tree using the probability sequence of the root node
        self.fill_probability_matrix(matrix)
        self.fill_probability_sequence()
        root_node = find_root_node(self.position)
        npmm = np.matmul(root_node.prob, np.array([0.25,0.25,0.25,0.25])) 
        log_likelihood = sum([math.log(npmm.flatten()[i]) for i in range(len(npmm.flatten()))]) # we calculate the log likelihood of the root node
        self.log_likelihood = log_likelihood # we add the log likelihood as an attribute of the tree
        return log_likelihood
    
    
        

In [5]:
tree_test = Tree(create_node_list(table, msa, brlen))
tree_test2 = Tree(create_node_list(table2, msa2, brlen2))
tree_test3 = Tree(create_node_list(table3, msa3, brlen3))
tree_test.tree_log_likelihood(Q)
tree_test2.tree_log_likelihood(Q)
tree_test3.tree_log_likelihood(Q)

print(tree_test.log_likelihood)   
print(tree_test2.log_likelihood)
print(tree_test3.log_likelihood)    # we print the log likelihood of the trees

-7053.683283562303
-7827.881832820439
-188.20007684036113
